<h1 align="center"> Generative AI Hackathon</h1>
<table align="center">
    <!-- <td>
        <a href="https://colab.research.google.com/github/teamdatatonic/gen-ai-hackathon/blob/feature/DBA-hackathon/notebook/analytics_hackathon.ipynb">
            <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo">
            <span style="vertical-align: middle;">Run in Colab</span>
        </a>
    </td> -->
    <!-- <td>
        <a href="https://github.com/teamdatatonic/gen-ai-hackathon/blob/DBA-hackathon/analytics_hackathon.ipynb">
            <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
            <span style="vertical-align: middle;">View on GitHub</span>
        </a>
    </td> -->
    <!-- <td>
        <a href="http://127.0.0.1:8888/?token=30f0873aab701a416cc3cc4be5926caa89940d3778fcef47
        ">
            <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"> 
            <span style="vertical-align: middle;">Open in Jupyter Notebook</span>
        </a>
    </td> -->
</table>
<hr>

**➡️ Your task:** Learn about Generative AI by building your own Analytics Assistant using Python and LangChain!

**❗ Note:** This workshop has been designed to be run in Jupyter Notebook. A credentials.json key will be shared with you for the purpose of running this project. 

Before getting started, let's first install some packages and dependencies

### Pip install package dependencies

In [ ]:
# %pip install --quiet "git+https://github.com/teamdatatonic/gen-ai-hackathon.git@feat/alvaro#egg=dt-gen-ai-analytics-helper"

In [1]:
!poetry install 
!poetry export --format requirements.txt --output requirements.txt
%pip install -r requirements.txt

Ignoring colorama: markers 'python_version >= "3.9" and python_version < "3.12" and (sys_platform == "win32" or platform_system == "Windows")' don't match your environment
Ignoring exceptiongroup: markers 'python_version >= "3.9" and python_version < "3.11"' don't match your environment
Ignoring tomli: markers 'python_version >= "3.9" and python_version < "3.11"' don't match your environment
Ignoring zipp: markers 'python_version >= "3.9" and python_version < "3.10"' don't match your environment

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


### Launch Jupyter Notebook

In [2]:
!poetry run jupyter notebook

Configuration file exists at /Users/alvaroazabal/Library/Preferences/pypoetry, reusing this directory.

Consider moving TOML configuration files to /Users/alvaroazabal/Library/Application Support/pypoetry, as support for the legacy directory will be removed in an upcoming release.
[I 2023-10-05 15:50:10.344 ServerApp] Package notebook took 0.0002s to import
[I 2023-10-05 15:50:10.469 ServerApp] Package jupyter_lsp took 0.1244s to import
[W 2023-10-05 15:50:10.470 ServerApp] A `_jupyter_server_extension_points` function was not found in jupyter_lsp. Instead, a `_jupyter_server_extension_paths` function was found and will be used for now. This function name will be deprecated in future releases of Jupyter Server.
[I 2023-10-05 15:50:10.530 ServerApp] Package jupyter_server_terminals took 0.0592s to import
[I 2023-10-05 15:50:10.533 ServerApp] Package jupyterlab took 0.0000s to import
[I 2023-10-05 15:50:13.451 ServerApp] Package notebook_shim took 0.0000s to import
[W 2023-10-05 15:50:13

**❗ Note:** This notebook will keep running until it is shut down manually.

## Analytics Assistant Hackathon - Start Here

### Vertex AI Endpoint

Currently, Vertex AI LLMs are accessible via Google Cloud projects. 

1. Set the env variables `project_id` and `dataset_id` with the filepath (**❗ Note:** the `/content/` folder is where uploaded files are stored by default).

In [1]:
# Replace 'your-project-id' with your Google Cloud project ID
PROJECT_ID = 'dt-gen-ai-hackathon-dev'
DATASET_ID = 'database_analytics_demo_v2'

In [ ]:
import os

# @title Set project credentials. { run: "auto", display-mode: "form" }
# @markdown Set the filepath to the `.json` credentials file.

GOOGLE_APPLICATION_CREDENTIALS = "credentials.json"  # @param {type:"string"}
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = GOOGLE_APPLICATION_CREDENTIALS

In [ ]:
!gcloud config set account dt-gen-ai-hackathon-sa@dt-gen-ai-hackathon-dev.iam.gserviceaccount.com
!gcloud auth activate-service-account --key-file={GOOGLE_APPLICATION_CREDENTIALS}
!gcloud config set project {PROJECT_ID}

### Import packages

In [3]:
from langchain.chains import SQLDatabaseSequentialChain
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import(create_pandas_dataframe_agent)
from sqlalchemy.ext.declarative import declarative_base
from langchain.agents.agent_types import AgentType
from langchain import LLMChain,PromptTemplate
from langchain.agents import create_sql_agent 
from sqlalchemy.engine import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from langchain.llms import VertexAI
from langchain import SQLDatabase
from tabulate import tabulate
from datetime import date
from pathlib import Path
import pandas as pd
import gradio as gr
import time

## Begin the Hackathon

### Objective

The objective of the hackathon is to show how to use Generative AI to query BigQuery tables and extract useful insights from it. This is done by leveraging the following technologies:
- PaLM: this is the Large Language Model (LLM) that generates the SQL code required to extract the data, as well as the text generated to provide an answer to the human
- LangChain: this is an open-source package that allows you to easily use LLMs. LangChain introduces to concept of chains and agents. Chains are an abstraction of LLMs that already uses pre-built prompts for given tasks. For example, in this notebook you will use the `SQLDatabaseSequentialChain` (more detail on this later), which essentially orchestrates a combintation of prompts and LLM calls in sequence until the correct answer is extracted.
- Gradio: this is friendly low-level UI tool, that allows users to create UIs and chatbots to interact with LLMs


Throoughout this notebook, there will be a sequence of tasks for you to run. Some tasks will already be completed and other tasks will be open as a challenge for you to implement.


### Why do we need an Analytics Assistant?

Companies have a lot of data stored in data warehouses, such as BigQuery. A lot of insights can be generated from such data (information abotu customers, products, business performance, marketing campaigns, etc). However, often this data is not accessible to everybody, as it requires an understanding of writing SQL code. 

This means that while data might be available, most users will not be able to access it, or will depends on skilled engineers to write the queries to gain access to data. Having an Analytics Assistant removes this bottleneck and accelerates the "time to insight" metric. 

Additionally, in many cases the answer to a query is not enough to provide the required insight, and multiple back-and-forth questions & answers are needed. An Analytics Assistant enables you to properly "chat" with your data, ask follow-up questions, and even do tasks like write emails summarising the findings. Essentially, it allows users to not just generate SQL but go beyond that and generate insights that come out of the SQL answer.

### What data do we have?

For this demo, we will be using some mock data that aims to simulate real data that many companies would have. The data available has the following schema:



## Task 0

Before getting started with tasks, let's do Task 0 to ensure we can connect to BigQuery and already run some LLMs to query the data in the most basic way.

We are going to do the following:
- Connect to the Bigquery Database
- Set up LLM Chain - this will be the `SQLDatabaseSequentialChain`
- Example query with the above chain to extract an answer

#### Create Bigquery Engine

We are going to leverage the `sqlalchemy` package to create an engine, which acts as a connector to our BigQuery data. For this, lets create a `BigQueryDatabase` class. As you will see in the next section, an instance of this class will be passed as an input to the LLM, and in that way the LLM knows which BigQuery dataset it has access to and can query.

In [4]:

class Database:
    """Create a connector to your BigQuery dataset"""
    def __init__(self, url: str, schema: str = None):
        print("creating db engine...")
        self.engine = self.create_engine(url)
        print("creating db session...")
        self.base = declarative_base()
        self.sessionmaker = sessionmaker(
            autocommit=True, autoflush=True, bind=self.engine
        )
        self.schema = schema
        print("creating db connection...")
        self.connect = self.engine.connect()

    def create_engine(self, url):
        return create_engine(url)

    @property
    def dialect(self) -> str:
        return self.engine.dialect.name

    def create_session(self):
        return self.sessionmaker()
    
    def create_connection(self):
        return  self.connect
    

class BigQueryDatabase(Database):
    def __init__(
        self,
        project_id=PROJECT_ID,
        dataset_id=DATASET_ID,
    ):
        super().__init__(f"bigquery://{project_id}/{dataset_id}")
        self.schema = dataset_id

#### Create LLM Chain

Now we can use LangChain and some of the pre-built chains that let you easily connect to an LLM and use it. The chain we are going to use is called `SQLDatabaseSequentialChain`. [put link to docs]

Let's analyse what this chain is doing under the hood, as it will help us understand how to leverage to improve the LLM performance. The most important two arguments that this accepts as an input are `llm` and `db`. The former refers to which LLM you want to use. LangChain offers abstractions to multiple LLMs, both open-source and proprietary. For this notebook we will use the `VertexAI` LLM. This offers you to choose any LLM which is part of the Vertex Model Garden derived from PalM. This can be any of:
- `text-bison` - text generation model, and any of its versions and tuned variants
- `code-bison` - LLM tailored to write code, and any of its tuned variants
- `chat-bison` - text generation model optimised for chat interactions
- `embeddings-gecko` - this model lets you embed text for future use

The `db` argument refers to the database that you want to query, in this case the `BigQueryDatabse` created beforehand.


Under the hood, the `SQLDatabaseSequentialChain` is doing the following things (once we execute it it will be more clear, so please refer to this section again):
- List all tables that belong to the given database it has access to
- Based on the user question and all tables available, the LLM will indentify which table (or combination of tables) it will need to query to generate the SQL code
- A BigQuery job will be executed to extract the schema of the tables that have been identified as relevant
- Given the user question and the table schemas, the LLM will try to generate the SQL code required to answer the question
- The SQL code will be executed in BigQuery and the BigQuery answer will be returned
- The user question and BigQuery answer will be used by the LLM to formulate an answer in natural language, which is then returned to the user

This six steps are being executed in just one line of code, orchestrated in this order by the LangChain chain and powered by the PaLM LLM.

Let's see it in action.

In [5]:
def create_basic_sql_chain(llm, db, question):
    """ Create a SQLDatabaseSequentialChain using the VertexAI LLM.
    """
    
    db_chain = SQLDatabaseSequentialChain.from_llm(
        llm,
        db,
        verbose=True,
        return_intermediate_steps=True,
    )

    output = db_chain(question)
    sql_query = output["intermediate_steps"][1]
    response = output["result"]
    
    return response, sql_query


Before executing the query we need to initialise the LLM from Vertex AI and the BigQuery database connector. 

In [6]:
# Initialize Vertex LLM
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID)

# Initialize Vertex LLM
llm = VertexAI(model_name='text-bison@001',
               temperature=0, max_output_tokens=1024)

# Initialise BigQuery datatbase connector
db = BigQueryDatabase(project_id=PROJECT_ID, dataset_id=DATASET_ID)
session = db.create_session()

conn = db.create_connection()

langchain_db = SQLDatabase(
    db.engine, schema=db.schema, sample_rows_in_table_info=0)

creating db engine...
creating db session...
creating db connection...


/var/folders/zk/smbk95753blcq47sdj4f5xkc0000gn/T/ipykernel_11026/3343946512.py:7: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  self.base = declarative_base()


In [7]:
# Define a function to query the SQLDBChain
def query_database(question, llm=llm ,db=langchain_db):
    
    # Call the SQLDBChain to get the answer based on the question
    answer, sql_query = create_basic_sql_chain(llm=llm, db=langchain_db, question=question)

    return answer


#### Let's run it

Try asking different questions and see what the LLM is doing under the hood

In [8]:
query_database('how many items did I sell in January?')



> Entering new SQLDatabaseSequentialChain chain...


/Users/alvaroazabal/Documents/gen-ai-hackathon/notebook/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Table names to use:
['orders']

> Entering new SQLDatabaseChain chain...
how many items did I sell in January?
SQLQuery:SELECT SUM(quantity) FROM `database_analytics_demo_v2`.`orders` WHERE order_date BETWEEN '2023-01-01' AND '2023-01-31'
SQLResult: [(38,)]
Answer:38
> Finished chain.

> Finished chain.


'38'

## Task 1 - Improve the Chain

#### Override default prompt

As you could see, sometimes the LLM does not return the correct answer, or tries to generate a syntantically incorrect query. Additionally, you really don't have control over which tables the LLM can and cannot use. Also, you want the LLM to be able to know the current date so that the questions such as "sales in the last three months" are always up to date. 

All of this can be done by overrriding the default prompt provided by LangChain in the `SQLDatabaseSequentialChain`. Here is an example of how to do. Next, you can play with this prompt and create your own prompt too.

In [9]:
CUSTOM_SQL_PROMPT = """
You are a GoogleSQL expert. Given an input question, first create a syntactically
correct GoogleSQL query to run, then look at the results of the query and return
the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain,
query for at most {top_k} results using the LIMIT clause as per GoogleSQL. You can
order the results to return the most informative data in the database.

Never query for all columns from a table. You must query only the columns that are
needed to answer the question. Wrap each column name and value in backticks (`)
to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below. Be careful
to not query for columns that do not exist. Also, pay attention to which column
is in which table.

Name all columns in the returned data appropriately. If a column does not have a
matching name in the schema, create an appropriate name reflecting its content.

Use the following format:

Question: "Question here"

SQLQuery: "SQL Query to run"

SQLResult: "Result of the SQLQuery"

Answer: "Final answer here"


Today''s date is {today_date}. When querying between dates, add the dates in quotes
('')

If someone asks for a specific month, use the range between the current month''s
start date and the current month''s end date.

If someone asks for a specific year, use the range between the first month of the
current year and the current month''s end date.


Remember to always use natural language when writing your final answer.

Only use the following tables:

{table_info}

Question: {question}

"""

In [10]:
from datetime import datetime

table_names = ["customers","employees","financial_goals","inventory","orders","product_reviews","supplier_orders"]

def create_sql_chain(question: str, table_info: str = table_names, top_k:int=100, llm: VertexAI = llm, db=langchain_db):
    """ Create a Q&A conversation chain using the VertexAI LLM.

    """
    
    db_chain = SQLDatabaseSequentialChain.from_llm(
        llm,
        db,
        verbose=True,
        return_intermediate_steps=True,
    )
    test_prompt = PromptTemplate(template=CUSTOM_SQL_PROMPT, input_variables=["question", "table_info", "today_date", "top_k"])

    today_date = datetime.now().strftime("%m/%d/%Y")
    output = db_chain(test_prompt.format(
        question=question,
        table_info=table_info,
        today_date=today_date,
        top_k=top_k
        ))
    sql_query = output["intermediate_steps"][1]
    response = output["result"]
    
    return response, sql_query


### Your task - Part 1 -> Create your own prompt and remove tables from the LLM

<<< TODO: Create a new prompt and replace the existing chain with that prompt>>>

In [11]:
TASK_1_PROMPT = """
You are a GoogleSQL expert. Given an input question, first create a syntactically
correct GoogleSQL query to run, then look at the results of the query and return
the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain,
query for at most {top_k} results using the LIMIT clause as per GoogleSQL. You can
order the results to return the most informative data in the database.

Never query for all columns from a table. You must query only the columns that are
needed to answer the question. Wrap each column name and value in backticks (`)
to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below. Be careful
to not query for columns that do not exist. Also, pay attention to which column
is in which table.

Name all columns in the returned data appropriately. If a column does not have a
matching name in the schema, create an appropriate name reflecting its content.

Use the following format:

Question: "Question here"

SQLQuery: "SQL Query to run"

SQLResult: "Result of the SQLQuery"

Answer: "Final answer here"


Today''s date is {today_date}. When querying between dates, add the dates in quotes
('')

If someone asks for a specific month, use the range between the current month''s
start date and the current month''s end date.

If someone asks for a specific year, use the range between the first month of the
current year and the current month''s end date.


Remember to always use natural language when writing your final answer.

Only use the following tables:

{table_info}

Question: {question}

"""

In [12]:
def create_sql_chain_task1(question: str, table_info: str = table_names, top_k:int=100, llm: VertexAI = llm, db=langchain_db):
    """ Create a Q&A conversation chain using the VertexAI LLM.

    """
    
    db_chain = SQLDatabaseSequentialChain.from_llm(
        llm,
        db,
        verbose=True,
        return_intermediate_steps=True,
    )
    test_prompt = PromptTemplate(template=TASK_1_PROMPT, input_variables=["question", "table_info", "today_date", "top_k"])

    today_date = datetime.now().strftime("%m/%d/%Y")
    output = db_chain(test_prompt.format(
        question=question,
        table_info=table_info,
        today_date=today_date,
        top_k=top_k
        ))
    sql_query = output["intermediate_steps"][1]
    response = output["result"]
    
    return response, sql_query


In [13]:
create_sql_chain_task1("how many items did I sell in January?")



> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['orders']

> Entering new SQLDatabaseChain chain...

You are a GoogleSQL expert. Given an input question, first create a syntactically
correct GoogleSQL query to run, then look at the results of the query and return
the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain,
query for at most 100 results using the LIMIT clause as per GoogleSQL. You can
order the results to return the most informative data in the database.

Never query for all columns from a table. You must query only the columns that are
needed to answer the question. Wrap each column name and value in backticks (`)
to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below. Be careful
to not query for columns that do not exist. Also, pay attention to which column
is in which table.

Name all columns in the returned data appropriate

('I sold 38 items in January.',
 "SELECT SUM(quantity) AS total_quantity_sold\nFROM `database_analytics_demo_v2`.`orders`\nWHERE order_date BETWEEN '2023-01-01' AND '2023-01-31'")

<<< TODO: Remove access to some tables>>>

In [10]:
table_names = ["customers","employees"]
langchain_db_subset = SQLDatabase(
    db.engine, schema=db.schema, sample_rows_in_table_info=0, include_tables=table_names)
create_sql_chain(question='how many items did I sell in January?', table_info=table_names, db=langchain_db_subset)



> Entering new SQLDatabaseSequentialChain chain...


/Users/alvaroazabal/.pyenv/versions/3.11.2/lib/python3.11/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Table names to use:
['orders']

> Entering new SQLDatabaseChain chain...

You are a GoogleSQL expert. Given an input question, first create a syntactically
correct GoogleSQL query to run, then look at the results of the query and return
the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain,
query for at most 100 results using the LIMIT clause as per GoogleSQL. You can
order the results to return the most informative data in the database.

Never query for all columns from a table. You must query only the columns that are
needed to answer the question. Wrap each column name and value in backticks (`)
to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below. Be careful
to not query for columns that do not exist. Also, pay attention to which column
is in which table.

Name all columns in the returned data appropriately. If a column does not have a
matching name in the 

('I sold 38 items in January.',
 "SELECT SUM(quantity) AS total_quantity_sold\nFROM `database_analytics_demo_v2`.`orders`\nWHERE order_date BETWEEN '2023-01-01' AND '2023-01-31'")

### Your task - Part 2 -> Chain multiple chains together for more powerful tasks

The beauty of LLMs and chains is that you can combine multiple chains together, each of them using different LLMs, to achieve more powerful tasks, such as automatically writing emails to sumamrise the answer returned by the SQL generation code.

For this task, first explore how to use the `LLMChain`, which is the simplest abstraction of a prompt + LLM pair. 

1. Implement the `LLMChain` and create a prompt to do any given task defined by you
2. Combine the existing `SQLDatabaseSequentialChain` with the `LLMChain` to create a more powerful model. Possible ideas are:
- Writing an email summarising the response
- Suggest three good follow-up questions that a user might want to ask

*Bonus track*: Can you try and use different LLMs for each of the tasks?

In [14]:
# Part 1 - Create a new LLMChain

# TODO: Create a prompt for this new chain
TASK_PROMPT = """You are an Analytics Assistant. Your task is to assist users to better understand the insights from their BigQuery datasets.
Users will ask a question to BigQuery and you will receive the question and the answer. 

Your task is to draft an email summarising the answer provided to the user.

The user question was:
```
{question}
```
and the answer provided was 
```
{answer}
```

Based on this, draft an email. Structure the email as follows:
1. Start with a cordial introduction
2. Remind the recipient as to what the user question was
3. Provide a summary of the answer
4. Send your best regards and say that you are happy to have a follow up

Answer here:

"""

task_prompt = PromptTemplate(template=TASK_PROMPT, input_variables=["question", "answer"])

# TODO: Complete the chain below
def get_task_chain(question, answer):
    
    task_chain = LLMChain(
        llm=llm, prompt=task_prompt, output_key="output")
    
    return task_chain.run(
        {
            "question": question,
            "answer": answer,
        }
    )




In [15]:
# TODO: Execute the new task chain to test it out
get_task_chain("what is your name?", "My name is Alvaro")

'Dear [recipient name],\n\nI hope this email finds you well.\n\nI am writing to you today to provide a summary of the answer to the user question:\n\n```\nwhat is your name?\n```\n\nThe answer provided by BigQuery was:\n\n```\nMy name is Alvaro\n```\n\nI hope this information is helpful. Please let me know if you have any further questions or need assistance.\n\nBest regards,\n[Your name]'

Now, let's see how to combine both chains to get an even more powerful model. To do so, we need to make sure we correctly get the output of the first chain (the `SQLDatabaseSequentialChain`) and pass it as input to the second chain (the `LLMChain`). Let's see how to do so.

In [16]:
 
def sql_and_task_chain(question, table_names=table_names):
   # Call the SQLDBSequentialChain to get the answer based on the question
    response, sql_query = create_sql_chain(question=question,table_info=table_names, db=langchain_db)

    chatbot_history = []

    chatbot_history.append(
        (
            question,
            response
        )
    )
   
    output = get_task_chain(question=question,
                           answer=response)
    
    chatbot_history.append(
        (
            output,
        )
    )
    
    return output

In [17]:
sql_and_task_chain(question='how many items did I sell in January?',table_names=table_names)



> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['orders']

> Entering new SQLDatabaseChain chain...

You are a GoogleSQL expert. Given an input question, first create a syntactically
correct GoogleSQL query to run, then look at the results of the query and return
the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain,
query for at most 100 results using the LIMIT clause as per GoogleSQL. You can
order the results to return the most informative data in the database.

Never query for all columns from a table. You must query only the columns that are
needed to answer the question. Wrap each column name and value in backticks (`)
to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below. Be careful
to not query for columns that do not exist. Also, pay attention to which column
is in which table.

Name all columns in the returned data appropriate

"Hi there,\n\nI hope this email finds you well.\n\nI'm writing to you today to follow up on your question about how many items you sold in January.\n\nI'm happy to report that I was able to find the answer to your question. You sold 38 items in January.\n\nI hope this information is helpful. Please let me know if you have any other questions. I'm happy to help.\n\nBest regards,\n\nYour Analytics Assistant"

## Task 2: Create Simple Gradio Interface 

You might have noticed that running LLMs in a notebook is not very user friendly nor interactive, and you cannot really chat with your data easily. 

To avoid this, let's see how to leverage Gradio to create a friendly UI that users can access to chat with their data.

We are going to start with a basic Gradio `interface` and then look at how to create a more complex chatbot that allows for multiple chat interactions.

#### Create a basic interface

In [18]:
# Create a Gradio interface
iface = gr.Interface(
    fn= sql_and_task_chain,  # Function to execute when a query is received
    inputs="text",      # Input is a single text field
    outputs="text",     # Output will be a text response
    title="Analytics Worker Demo",
    description="Enter a question, and the system will query the database and provide an answer.",
)

# Launch the Gradio interface on a specified port (e.g., 5000)
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://ed2dfca70e38f93280.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new SQLDatabaseSequentialChain chain...


/Users/alvaroazabal/Documents/gen-ai-hackathon/notebook/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Table names to use:
['orders']

> Entering new SQLDatabaseChain chain...

You are a GoogleSQL expert. Given an input question, first create a syntactically
correct GoogleSQL query to run, then look at the results of the query and return
the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain,
query for at most 100 results using the LIMIT clause as per GoogleSQL. You can
order the results to return the most informative data in the database.

Never query for all columns from a table. You must query only the columns that are
needed to answer the question. Wrap each column name and value in backticks (`)
to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below. Be careful
to not query for columns that do not exist. Also, pay attention to which column
is in which table.

Name all columns in the returned data appropriately. If a column does not have a
matching name in the 

#### Now let's create an actual chatbot

In [19]:

# Gradio chatbot and interface
with gr.Blocks(title="Analytics Assistant") as demo:

    with gr.Row():
                with gr.Column(scale=1, variant="panel"):
                    with gr.Row():
                        # Load Datatonic logo as .svg
                        gr.Markdown(
                            """\
<svg width="177" height="24" viewBox="0 0 177 24" xmlns="http://www.w3.org/2000/svg"><path d="M14.5548 14.596H9.37748V9.404H0V14.596H9.37748V24H14.5548V14.596H23.9323V9.404H14.5548V14.596Z" fill="#2a5cff"></path><path d="M14.5548 0H9.37748V9.404H14.5548V0Z" fill="#2a5cff"></path><path d="M59.6168 6.3732C55.4116 6.3732 52.3637 8.42451 51.9375 11.8749H56.6305C56.8133 10.6942 57.7879 9.60525 59.6168 9.60525C61.5972 9.60525 62.5728 10.9415 62.5728 12.6518V12.931L58.2151 13.3658C54.9541 13.6769 51.3588 14.6403 51.3588 18.4955C51.3588 21.7594 53.918 23.9981 57.1183 23.9981C60.3186 23.9981 61.6891 22.4753 62.6334 20.454V23.6252H67.2345V12.9938C67.2345 9.20136 64.7663 6.37226 59.6168 6.37226V6.3732ZM62.5728 16.4451C62.5728 19.0877 61.3235 21.139 58.9462 21.139C57.3617 21.139 56.1124 20.1448 56.1124 18.4964C56.1124 16.5997 58.0323 16.2586 59.9218 16.0403L62.5728 15.7601V16.4451Z" fill="#2a5cff"></path><path d="M76.7895 10.2275H80.416V6.74523H76.7895V2.76725H72.1277V6.74616H69.0496V10.2284H72.1277V18.622C72.1277 21.9796 74.5656 23.627 77.8257 23.627H80.4463V20.1457H78.8315C77.4904 20.1457 76.7895 19.7109 76.7895 18.249V10.2275Z" fill="#2a5cff"></path><path d="M90.6838 6.3732C86.4786 6.3732 83.4308 8.42451 83.0046 11.8749H87.6975C87.8803 10.6942 88.8549 9.60525 90.6838 9.60525C92.6643 9.60525 93.6398 10.9415 93.6398 12.6518V12.931L89.2821 13.3658C86.0212 13.6769 82.4259 14.6403 82.4259 18.4955C82.4259 21.7594 84.985 23.9981 88.1853 23.9981C91.3856 23.9981 92.7561 22.4753 93.7004 20.454V23.6252H98.3016V12.9938C98.3016 9.20136 95.8333 6.37226 90.6838 6.37226V6.3732ZM93.6398 16.4451C93.6398 19.0877 92.3905 21.139 90.0133 21.139C88.4287 21.139 87.1795 20.1448 87.1795 18.4964C87.1795 16.5997 89.0993 16.2586 90.9888 16.0403L93.6398 15.7601V16.4451Z" fill="#2a5cff"></path><path d="M108.022 10.2275H111.648V6.74523H108.022V2.76725H103.36V6.74616H100.282V10.2284H103.36V18.622C103.36 21.9796 105.798 23.627 109.058 23.627H111.679V20.1457H110.064C108.723 20.1457 108.022 19.7109 108.022 18.249V10.2275Z" fill="#2a5cff"></path><path d="M121.926 6.3732C116.624 6.3732 113.303 10.0101 113.303 15.2016C113.303 20.3931 116.624 23.9991 121.926 23.9991C127.228 23.9991 130.55 20.3622 130.55 15.2016C130.55 10.041 127.228 6.3732 121.926 6.3732ZM121.926 20.7661C119.397 20.7661 118.056 18.6211 118.056 15.2016C118.056 11.7821 119.397 9.60618 121.926 9.60618C124.455 9.60618 125.796 11.7512 125.796 15.2016C125.796 18.652 124.455 20.7661 121.926 20.7661Z" fill="#2a5cff"></path><path d="M143.121 6.3732C140.226 6.3732 138.61 8.08246 137.849 10.1966V6.74616H133.217V23.6261H137.88V14.0199C137.88 11.0353 139.007 9.69896 140.927 9.69896C142.847 9.69896 143.974 11.0353 143.974 13.9581V23.6261H148.637V12.9319C148.637 9.20136 146.991 6.3732 143.121 6.3732H143.121Z" fill="#2a5cff"></path><path d="M157.046 6.74616H152.383V23.6261H157.046V6.74616Z" fill="#2a5cff"></path><path d="M157.137 0H152.323V4.47651H157.137V0Z" fill="#2a5cff"></path><path d="M172.107 17.6268C171.589 19.5853 170.492 20.767 168.572 20.767C166.165 20.767 164.763 18.7148 164.763 15.1716C164.763 11.6284 166.104 9.60712 168.572 9.60712C170.492 9.60712 171.559 11.0371 171.955 12.8092H176.647C175.947 8.86119 172.93 6.37414 168.572 6.37414C163.514 6.37414 160.009 9.88731 160.009 15.1716C160.009 20.4559 163.453 24 168.572 24C172.717 24 176.007 21.7613 176.8 17.6268H172.107Z" fill="#2a5cff"></path><path d="M37.2505 0H28.7188V9.26789H33.533V4.10355H37.3727C41.6082 4.10355 44.0764 6.83894 44.0764 11.813C44.0764 16.7872 41.6082 19.5226 37.3727 19.5226H33.533V14.3666H28.7188V23.6261H37.2505C44.2895 23.6261 49.0431 19.7718 49.0431 11.813C49.0431 3.85428 44.2895 0 37.2505 0Z" fill="#2a5cff"></path><path d="M38.5356 9.26789H33.5376V14.3666H38.5356V9.26789Z" fill="#2a5cff"></path></svg>"""
                        )
                        gr.Markdown(
                            "# Datatonic Analytics Assistant",
                            elem_classes="title right",
                        )

    chatbot = gr.Chatbot()

    with gr.Tab("Ask a question:"):
        # Create a textbox for user questions
        msg = gr.Textbox(show_label=False)

        with gr.Row().style(equal_height=False):
            with gr.Column(scale=3):
                with gr.Row():
                    send_message = gr.Button(
                        value="Submit", variant="primary"
                    ).style(size="sm")
                    clear = gr.ClearButton([msg, chatbot])

    def respond(question, chat_history):
        bot_message, _ = create_sql_chain(question)
        chat_history.append((question, bot_message))
        time.sleep(2)
        return bot_message, chat_history
    

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    send_message.click(
                respond, [msg, chatbot], [msg, chatbot])
    

demo.launch(share=True)

/var/folders/zk/smbk95753blcq47sdj4f5xkc0000gn/T/ipykernel_11026/1202885413.py:23: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  with gr.Row().style(equal_height=False):
/var/folders/zk/smbk95753blcq47sdj4f5xkc0000gn/T/ipykernel_11026/1202885413.py:28: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  ).style(size="sm")


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://1fad45d36670b1dfa2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new SQLDatabaseSequentialChain chain...


/Users/alvaroazabal/Documents/gen-ai-hackathon/notebook/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Table names to use:
['orders']

> Entering new SQLDatabaseChain chain...

You are a GoogleSQL expert. Given an input question, first create a syntactically
correct GoogleSQL query to run, then look at the results of the query and return
the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain,
query for at most 100 results using the LIMIT clause as per GoogleSQL. You can
order the results to return the most informative data in the database.

Never query for all columns from a table. You must query only the columns that are
needed to answer the question. Wrap each column name and value in backticks (`)
to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below. Be careful
to not query for columns that do not exist. Also, pay attention to which column
is in which table.

Name all columns in the returned data appropriately. If a column does not have a
matching name in the 

### Task -> Use chatbot history for LLM query

Notice how although we are using the `gr.Chatbot` component, the chat history is not being passed to the LLM, which means the model is not really a chatbot

Explain the process workflow

In [20]:
from datetime import datetime
import json 

table_names = ["customers","employees","financial_goals","inventory","orders","product_reviews","supplier_orders"]

PROMPT_WITH_HISTORY = """You are a analytics assistant that understands human natural language business related questions and are able to convert this into relevant SQL code or answer the questions provided the relevant chat history is available.
The user will write a prompt asking for you to generate an answer based on a BigQuery table.

You will receive as an input the user question, the past chat history and a list of table_names.

You have two tasks.
First, you need to understand if the user question and chat history requires you to generate SQL code or if you just need to answer the question directly.
Secondly, if the user inputs do not require SQL code to be generated then you will need to answer the user question using any relevant information provided in the chat history.

Output a JSON to identify if SQL is required and the direct answer if SQL is not required. The JSON schema format is:
'{{
    "requires_sql": boolean \\ Value can be either true or false
    "direct_answer": string \\ NOTE: only include this attribute if SQL code is not required
}}'

REMEMBER: "require_sql" is a boolean therefore can only be true or false
REMEMBER: if require_sql is false then answer using the information in the chat history and user question only. DO NOT MAKE UP INFORMATION. If you don't have the necessary information to provide an answer, return "I don't know, please ask the question differently".

You have access to the following past chat history:

<< CHAT HISTORY >>
{chat_history}

You have access to the following BigQuery Tables:

<< BigQuery TABLES >>
{table_info}

The user query is:

<< USER QUESTION >>
{question}

Based on the query and chat history, return the answer below in the following JSON format:
'{{
    "requires_sql": boolean \\ Value can be either true or false
    "direct_answer": string \\ NOTE: only include this attribute if SQL code is not required
}}'


<< EXAMPLES >>
Chat History: ["Hello how can i help you today?"]
BigQuery Tables: ['customers','employees','financial_goals','inventory','orders','product_reviews','supplier_orders']
User question: What is revenue in 2023?
Output JSON:'{{
    "requires_sql": true,
    "direct_answer": ""
}}'

Chat History: ["Revenue is down 20 points last month"]
BigQuery Tables: [customers,orders,employees,inventory,product_reviews,supplier_orders,financial_goal]
User question: Is this month better than last?
Output JSON:'{{
    "requires_sql": false,
    "direct_answer": "No, this month is worse as revenue is down by 20 points"
}}'


<< ANSWER >>
"""

def create_sql_chain_with_history(question: str, history:str, table_info: str = table_names, top_k:int=100, llm: VertexAI = llm, db=langchain_db):
    """ Create a Q&A conversation chain using the VertexAI LLM.

    """

    chat_prompt = PromptTemplate(template=PROMPT_WITH_HISTORY, input_variables=["question", "table_info","chat_history"])

    chat_chain = LLMChain(
            llm=llm, prompt=chat_prompt, output_key="output")
        
    output = chat_chain.run(
        {
            "question": question,
            "chat_history": history,
            "table_info": table_info
        }
    )

    json_output = json.loads(output)
    
    if json_output["requires_sql"] == True:
        response, _ = create_sql_chain(question)
    elif json_output["requires_sql"] == False:
        response = json_output["direct_answer"]
    else:
        response = "Sorry but I could not identify your question, please ask again"

    
    return response

Notice how inside the `respond` function we are creating a chatbot history and append every question and answer to it. However, this history is not being used as it is not passed as an input to the LLM chain. 

Your task is to modify the chain so that it accepts as an input the history and you can properly chat with your data with a model that contains memory and remembers past questions.

In [21]:
# TODO: Add chat history to chain

# Gradio chatbot and interface
with gr.Blocks(title="Analytics Assistant") as demo:

    with gr.Row():
                with gr.Column(scale=1, variant="panel"):
                    with gr.Row():
                        # Load Datatonic logo as .svg
                        gr.Markdown(
                            """\
<svg width="177" height="24" viewBox="0 0 177 24" xmlns="http://www.w3.org/2000/svg"><path d="M14.5548 14.596H9.37748V9.404H0V14.596H9.37748V24H14.5548V14.596H23.9323V9.404H14.5548V14.596Z" fill="#2a5cff"></path><path d="M14.5548 0H9.37748V9.404H14.5548V0Z" fill="#2a5cff"></path><path d="M59.6168 6.3732C55.4116 6.3732 52.3637 8.42451 51.9375 11.8749H56.6305C56.8133 10.6942 57.7879 9.60525 59.6168 9.60525C61.5972 9.60525 62.5728 10.9415 62.5728 12.6518V12.931L58.2151 13.3658C54.9541 13.6769 51.3588 14.6403 51.3588 18.4955C51.3588 21.7594 53.918 23.9981 57.1183 23.9981C60.3186 23.9981 61.6891 22.4753 62.6334 20.454V23.6252H67.2345V12.9938C67.2345 9.20136 64.7663 6.37226 59.6168 6.37226V6.3732ZM62.5728 16.4451C62.5728 19.0877 61.3235 21.139 58.9462 21.139C57.3617 21.139 56.1124 20.1448 56.1124 18.4964C56.1124 16.5997 58.0323 16.2586 59.9218 16.0403L62.5728 15.7601V16.4451Z" fill="#2a5cff"></path><path d="M76.7895 10.2275H80.416V6.74523H76.7895V2.76725H72.1277V6.74616H69.0496V10.2284H72.1277V18.622C72.1277 21.9796 74.5656 23.627 77.8257 23.627H80.4463V20.1457H78.8315C77.4904 20.1457 76.7895 19.7109 76.7895 18.249V10.2275Z" fill="#2a5cff"></path><path d="M90.6838 6.3732C86.4786 6.3732 83.4308 8.42451 83.0046 11.8749H87.6975C87.8803 10.6942 88.8549 9.60525 90.6838 9.60525C92.6643 9.60525 93.6398 10.9415 93.6398 12.6518V12.931L89.2821 13.3658C86.0212 13.6769 82.4259 14.6403 82.4259 18.4955C82.4259 21.7594 84.985 23.9981 88.1853 23.9981C91.3856 23.9981 92.7561 22.4753 93.7004 20.454V23.6252H98.3016V12.9938C98.3016 9.20136 95.8333 6.37226 90.6838 6.37226V6.3732ZM93.6398 16.4451C93.6398 19.0877 92.3905 21.139 90.0133 21.139C88.4287 21.139 87.1795 20.1448 87.1795 18.4964C87.1795 16.5997 89.0993 16.2586 90.9888 16.0403L93.6398 15.7601V16.4451Z" fill="#2a5cff"></path><path d="M108.022 10.2275H111.648V6.74523H108.022V2.76725H103.36V6.74616H100.282V10.2284H103.36V18.622C103.36 21.9796 105.798 23.627 109.058 23.627H111.679V20.1457H110.064C108.723 20.1457 108.022 19.7109 108.022 18.249V10.2275Z" fill="#2a5cff"></path><path d="M121.926 6.3732C116.624 6.3732 113.303 10.0101 113.303 15.2016C113.303 20.3931 116.624 23.9991 121.926 23.9991C127.228 23.9991 130.55 20.3622 130.55 15.2016C130.55 10.041 127.228 6.3732 121.926 6.3732ZM121.926 20.7661C119.397 20.7661 118.056 18.6211 118.056 15.2016C118.056 11.7821 119.397 9.60618 121.926 9.60618C124.455 9.60618 125.796 11.7512 125.796 15.2016C125.796 18.652 124.455 20.7661 121.926 20.7661Z" fill="#2a5cff"></path><path d="M143.121 6.3732C140.226 6.3732 138.61 8.08246 137.849 10.1966V6.74616H133.217V23.6261H137.88V14.0199C137.88 11.0353 139.007 9.69896 140.927 9.69896C142.847 9.69896 143.974 11.0353 143.974 13.9581V23.6261H148.637V12.9319C148.637 9.20136 146.991 6.3732 143.121 6.3732H143.121Z" fill="#2a5cff"></path><path d="M157.046 6.74616H152.383V23.6261H157.046V6.74616Z" fill="#2a5cff"></path><path d="M157.137 0H152.323V4.47651H157.137V0Z" fill="#2a5cff"></path><path d="M172.107 17.6268C171.589 19.5853 170.492 20.767 168.572 20.767C166.165 20.767 164.763 18.7148 164.763 15.1716C164.763 11.6284 166.104 9.60712 168.572 9.60712C170.492 9.60712 171.559 11.0371 171.955 12.8092H176.647C175.947 8.86119 172.93 6.37414 168.572 6.37414C163.514 6.37414 160.009 9.88731 160.009 15.1716C160.009 20.4559 163.453 24 168.572 24C172.717 24 176.007 21.7613 176.8 17.6268H172.107Z" fill="#2a5cff"></path><path d="M37.2505 0H28.7188V9.26789H33.533V4.10355H37.3727C41.6082 4.10355 44.0764 6.83894 44.0764 11.813C44.0764 16.7872 41.6082 19.5226 37.3727 19.5226H33.533V14.3666H28.7188V23.6261H37.2505C44.2895 23.6261 49.0431 19.7718 49.0431 11.813C49.0431 3.85428 44.2895 0 37.2505 0Z" fill="#2a5cff"></path><path d="M38.5356 9.26789H33.5376V14.3666H38.5356V9.26789Z" fill="#2a5cff"></path></svg>"""
                        )
                        gr.Markdown(
                            "# Datatonic Analytics Assistant",
                            elem_classes="title right",
                        )

    chatbot = gr.Chatbot()

    with gr.Tab("Ask a question:"):
        # Create a textbox for user questions
        msg = gr.Textbox(show_label=False)

        with gr.Row().style(equal_height=False):
            with gr.Column(scale=3):
                with gr.Row():
                    send_message = gr.Button(
                        value="Submit", variant="primary"
                    ).style(size="sm")
                    clear = gr.ClearButton([msg, chatbot])

    def respond(question, chat_history):
        bot_message = create_sql_chain_with_history(question=question, history=chat_history)
        chat_history.append((question, bot_message))
        time.sleep(2)
        return bot_message, chat_history
    

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    send_message.click(
                respond, [msg, chatbot], [msg, chatbot])
    

demo.launch(share=True)

/var/folders/zk/smbk95753blcq47sdj4f5xkc0000gn/T/ipykernel_11026/551329854.py:25: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  with gr.Row().style(equal_height=False):
/var/folders/zk/smbk95753blcq47sdj4f5xkc0000gn/T/ipykernel_11026/551329854.py:30: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  ).style(size="sm")


Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://dad6c94aa0e40ff76e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new SQLDatabaseSequentialChain chain...


/Users/alvaroazabal/Documents/gen-ai-hackathon/notebook/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Table names to use:
['orders']

> Entering new SQLDatabaseChain chain...

You are a GoogleSQL expert. Given an input question, first create a syntactically
correct GoogleSQL query to run, then look at the results of the query and return
the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain,
query for at most 100 results using the LIMIT clause as per GoogleSQL. You can
order the results to return the most informative data in the database.

Never query for all columns from a table. You must query only the columns that are
needed to answer the question. Wrap each column name and value in backticks (`)
to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below. Be careful
to not query for columns that do not exist. Also, pay attention to which column
is in which table.

Name all columns in the returned data appropriately. If a column does not have a
matching name in the 

## Task 3: Create SQL Agent with ToolKit

Now that we have a more capable LLM and Gradio interface implemented, can we make it better?

One alternative to chains are agents. Whereas chains have a very well defined workflow with clear steps to follow and a start and end, LLM agents are given more freedom to try different actions and go back to the starting point and try again different methods. This is done by using the `REACT` framework and providing LLMs with toolkits. What does this mean? 

First, let's looka at toolkits. A toolkit is, as the word implies, an external capability that you provide to the LLM. A potential toolkit might be the Google Search API. If the LLM agent is given this toolkit, the LLM could access the Google Search API to search for information in Google. Another toolkit is the SQL toolkit (used here) which gives it acces to a SQL database. Other common toolkits are the python REPL (that allows the LLM to execute python code) or a calculator, to do mathematical calculations.

Now, let's understand the `REACT` framework. This essentially is a way that defines how an LLM agent should "think". While an LLM agent is given access to a tool and given the freedom to use it, the `REACT` framework (which is essentially just a prompt) tells the model what steps it should follow to know if it should use one tool or another, or just return an answer. This works by ensuring the LLM always follows the same four steps until it reaches a final answer:
[use example from docs here] 

As you can see here, this means LLM agents are more powerful than chains, as they can do more actions the more tools you give it, but at the same time can be harder to control. Whether you use a chain or an agent is a trade-off that depends on each use case.

Let's see how to create a SQL agent.

In [20]:
def create_agent(llm, db, question):
    # Define our agent’s toolkit which will be used to answer the user question
    toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    qa_chain = create_sql_agent(
    llm=llm,
    db=db, 
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    early_stopping_method="generate")

    answer = qa_chain.run(question)

    return answer


# Define an sql agent with SQLToolKit
def sql_agent(question):
    response = create_agent(llm=llm, db=langchain_db, question=question)

    return response[0:]

In [21]:
sql_agent("how many items did I sell in January 2023?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: customers, employees, errors, feedback, financial_goals, history, inventory, marketing_campaign_analytics, orders, product_reviews, prompts, prompts_resummarised, queries, supplier_orders
Thought:The orders table seems relevant. I should query the schema of the orders table.
Action: sql_db_schema
Action Input: orders
Observation: 
CREATE TABLE `database_analytics_demo_v2`.`orders` (
	`order_id` INT64, 
	`customer_id` INT64, 
	`customer_first_name` STRING, 
	`customer_last_name` STRING, 
	`customer_email` STRING, 
	`phone_number` STRING, 
	`customer_street_address` STRING, 
	`customer_city` STRING, 
	`customer_state` STRING, 
	`customer_zip_code` STRING, 
	`product_id` INT64, 
	`product_name` STRING, 
	`product_description` STRING, 
	`product_category` STRING, 
	`product_price` FLOAT64, 
	`supplier_name` STRING, 
	`order_date` DATE, 
	`total_price` FLOAT64, 
	`shipping_address` STRING, 
	`orde

'38 items were sold in January 2023.'

#### Task -> Use the agent in the Gradio chatbot

In [90]:
# TODO: replace the chain with the agent in the Gradio code

# Gradio chatbot and interface
with gr.Blocks(title="Analytics Assistant") as demo:

    with gr.Row():
                with gr.Column(scale=1, variant="panel"):
                    with gr.Row():
                        # Load Datatonic logo as .svg
                        gr.Markdown(
                            """\
<svg width="177" height="24" viewBox="0 0 177 24" xmlns="http://www.w3.org/2000/svg"><path d="M14.5548 14.596H9.37748V9.404H0V14.596H9.37748V24H14.5548V14.596H23.9323V9.404H14.5548V14.596Z" fill="#2a5cff"></path><path d="M14.5548 0H9.37748V9.404H14.5548V0Z" fill="#2a5cff"></path><path d="M59.6168 6.3732C55.4116 6.3732 52.3637 8.42451 51.9375 11.8749H56.6305C56.8133 10.6942 57.7879 9.60525 59.6168 9.60525C61.5972 9.60525 62.5728 10.9415 62.5728 12.6518V12.931L58.2151 13.3658C54.9541 13.6769 51.3588 14.6403 51.3588 18.4955C51.3588 21.7594 53.918 23.9981 57.1183 23.9981C60.3186 23.9981 61.6891 22.4753 62.6334 20.454V23.6252H67.2345V12.9938C67.2345 9.20136 64.7663 6.37226 59.6168 6.37226V6.3732ZM62.5728 16.4451C62.5728 19.0877 61.3235 21.139 58.9462 21.139C57.3617 21.139 56.1124 20.1448 56.1124 18.4964C56.1124 16.5997 58.0323 16.2586 59.9218 16.0403L62.5728 15.7601V16.4451Z" fill="#2a5cff"></path><path d="M76.7895 10.2275H80.416V6.74523H76.7895V2.76725H72.1277V6.74616H69.0496V10.2284H72.1277V18.622C72.1277 21.9796 74.5656 23.627 77.8257 23.627H80.4463V20.1457H78.8315C77.4904 20.1457 76.7895 19.7109 76.7895 18.249V10.2275Z" fill="#2a5cff"></path><path d="M90.6838 6.3732C86.4786 6.3732 83.4308 8.42451 83.0046 11.8749H87.6975C87.8803 10.6942 88.8549 9.60525 90.6838 9.60525C92.6643 9.60525 93.6398 10.9415 93.6398 12.6518V12.931L89.2821 13.3658C86.0212 13.6769 82.4259 14.6403 82.4259 18.4955C82.4259 21.7594 84.985 23.9981 88.1853 23.9981C91.3856 23.9981 92.7561 22.4753 93.7004 20.454V23.6252H98.3016V12.9938C98.3016 9.20136 95.8333 6.37226 90.6838 6.37226V6.3732ZM93.6398 16.4451C93.6398 19.0877 92.3905 21.139 90.0133 21.139C88.4287 21.139 87.1795 20.1448 87.1795 18.4964C87.1795 16.5997 89.0993 16.2586 90.9888 16.0403L93.6398 15.7601V16.4451Z" fill="#2a5cff"></path><path d="M108.022 10.2275H111.648V6.74523H108.022V2.76725H103.36V6.74616H100.282V10.2284H103.36V18.622C103.36 21.9796 105.798 23.627 109.058 23.627H111.679V20.1457H110.064C108.723 20.1457 108.022 19.7109 108.022 18.249V10.2275Z" fill="#2a5cff"></path><path d="M121.926 6.3732C116.624 6.3732 113.303 10.0101 113.303 15.2016C113.303 20.3931 116.624 23.9991 121.926 23.9991C127.228 23.9991 130.55 20.3622 130.55 15.2016C130.55 10.041 127.228 6.3732 121.926 6.3732ZM121.926 20.7661C119.397 20.7661 118.056 18.6211 118.056 15.2016C118.056 11.7821 119.397 9.60618 121.926 9.60618C124.455 9.60618 125.796 11.7512 125.796 15.2016C125.796 18.652 124.455 20.7661 121.926 20.7661Z" fill="#2a5cff"></path><path d="M143.121 6.3732C140.226 6.3732 138.61 8.08246 137.849 10.1966V6.74616H133.217V23.6261H137.88V14.0199C137.88 11.0353 139.007 9.69896 140.927 9.69896C142.847 9.69896 143.974 11.0353 143.974 13.9581V23.6261H148.637V12.9319C148.637 9.20136 146.991 6.3732 143.121 6.3732H143.121Z" fill="#2a5cff"></path><path d="M157.046 6.74616H152.383V23.6261H157.046V6.74616Z" fill="#2a5cff"></path><path d="M157.137 0H152.323V4.47651H157.137V0Z" fill="#2a5cff"></path><path d="M172.107 17.6268C171.589 19.5853 170.492 20.767 168.572 20.767C166.165 20.767 164.763 18.7148 164.763 15.1716C164.763 11.6284 166.104 9.60712 168.572 9.60712C170.492 9.60712 171.559 11.0371 171.955 12.8092H176.647C175.947 8.86119 172.93 6.37414 168.572 6.37414C163.514 6.37414 160.009 9.88731 160.009 15.1716C160.009 20.4559 163.453 24 168.572 24C172.717 24 176.007 21.7613 176.8 17.6268H172.107Z" fill="#2a5cff"></path><path d="M37.2505 0H28.7188V9.26789H33.533V4.10355H37.3727C41.6082 4.10355 44.0764 6.83894 44.0764 11.813C44.0764 16.7872 41.6082 19.5226 37.3727 19.5226H33.533V14.3666H28.7188V23.6261H37.2505C44.2895 23.6261 49.0431 19.7718 49.0431 11.813C49.0431 3.85428 44.2895 0 37.2505 0Z" fill="#2a5cff"></path><path d="M38.5356 9.26789H33.5376V14.3666H38.5356V9.26789Z" fill="#2a5cff"></path></svg>"""
                        )
                        gr.Markdown(
                            "# Datatonic Analytics Assistant",
                            elem_classes="title right",
                        )

    chatbot = gr.Chatbot()

    with gr.Tab("Ask a question:"):
        # Create a textbox for user questions
        msg = gr.Textbox(show_label=False)

        with gr.Row().style(equal_height=False):
            with gr.Column(scale=3):
                with gr.Row():
                    send_message = gr.Button(
                        value="Submit", variant="primary"
                    ).style(size="sm")
                    clear = gr.ClearButton([msg, chatbot])
                    
    def respond(question, chat_history):
        bot_message = sql_agent(question)
        chat_history.append((question, bot_message))
        time.sleep(2)
        return bot_message, chat_history
    

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    send_message.click(
                respond, [msg, chatbot], [msg, chatbot])
    

demo.launch(share=True)

/var/folders/zk/smbk95753blcq47sdj4f5xkc0000gn/T/ipykernel_8063/3435402721.py:25: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  with gr.Row().style(equal_height=False):
/var/folders/zk/smbk95753blcq47sdj4f5xkc0000gn/T/ipykernel_8063/3435402721.py:30: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  ).style(size="sm")


Running on local URL:  http://127.0.0.1:7870
Running on public URL: https://f9389d15f0430b340c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: customers, employees, errors, feedback, financial_goals, history, inventory, marketing_campaign_analytics, orders, product_reviews, prompts, prompts_resummarised, queries, supplier_orders
Thought:The customers table seems relevant. I should query the schema of the customers table.
Action: sql_db_schema
Action Input: customers
Observation: 
CREATE TABLE `database_analytics_demo_v2`.`customers` (
	`customer_id` INT64, 
	`first_name` STRING, 
	`last_name` STRING, 
	`email` STRING, 
	`gender` STRING, 
	`phone_number` STRING, 
	`city` STRING, 
	`state` STRING, 
	`zipcode` STRING, 
	`country` STRING
)
Thought:The customers table has a customer_id column. I can query the number of customers by counting the number of rows in the customers table.
Action: sql_db_query
Action Input: SELECT count(*) FROM customers
Observation: [(1000,)]
Thought:I now know the final answer
Final Answer: There are 1000 cus

#### Task -> Add memory to the SQL Agent

Hint: Follow this page https://python.langchain.com/docs/modules/memory/agent_with_memory_in_db

## Bonus Track: Create Pandas Agent

If you've reached this step, well done! 

Now that we have made an SQLAgent and a Gradio chatbot, we can go further and make a Data Analytics agent that is able to perform analysis and plot relevant charts using pandas. For this, we can create an agent that also uses a SQL Chain, loads the answer of the chain to a pandas dataframe and then uses the pandas toolkit to solve the user query.

Try to implement it below.

*Hint*: Have a look at the `create_pandas_dataframe_agent`

In [91]:
def pandas_agent(question,llm, df):
    
    agent = create_pandas_dataframe_agent(llm, df, verbose=True)
    response = agent.run(question)
    return response

def pd_sql_agent(question):

    answer, sql_query = create_sql_chain(llm=llm, db=langchain_db, question=question)
    df = pd.read_sql(sql_query, conn, index_col=None)
 
    response = pandas_agent(question=question, llm=llm, df=df)

    # check if response is a plot, then render that in UI else return response
    return response

#### Test it out in Gradio

In [92]:
# Gradio chatbot and interface
with gr.Blocks(title="Analytics Assistant") as demo:

    with gr.Row():
                with gr.Column(scale=1, variant="panel"):
                    with gr.Row():
                        # Load Datatonic logo as .svg
                        gr.Markdown(
                            """\
<svg width="177" height="24" viewBox="0 0 177 24" xmlns="http://www.w3.org/2000/svg"><path d="M14.5548 14.596H9.37748V9.404H0V14.596H9.37748V24H14.5548V14.596H23.9323V9.404H14.5548V14.596Z" fill="#2a5cff"></path><path d="M14.5548 0H9.37748V9.404H14.5548V0Z" fill="#2a5cff"></path><path d="M59.6168 6.3732C55.4116 6.3732 52.3637 8.42451 51.9375 11.8749H56.6305C56.8133 10.6942 57.7879 9.60525 59.6168 9.60525C61.5972 9.60525 62.5728 10.9415 62.5728 12.6518V12.931L58.2151 13.3658C54.9541 13.6769 51.3588 14.6403 51.3588 18.4955C51.3588 21.7594 53.918 23.9981 57.1183 23.9981C60.3186 23.9981 61.6891 22.4753 62.6334 20.454V23.6252H67.2345V12.9938C67.2345 9.20136 64.7663 6.37226 59.6168 6.37226V6.3732ZM62.5728 16.4451C62.5728 19.0877 61.3235 21.139 58.9462 21.139C57.3617 21.139 56.1124 20.1448 56.1124 18.4964C56.1124 16.5997 58.0323 16.2586 59.9218 16.0403L62.5728 15.7601V16.4451Z" fill="#2a5cff"></path><path d="M76.7895 10.2275H80.416V6.74523H76.7895V2.76725H72.1277V6.74616H69.0496V10.2284H72.1277V18.622C72.1277 21.9796 74.5656 23.627 77.8257 23.627H80.4463V20.1457H78.8315C77.4904 20.1457 76.7895 19.7109 76.7895 18.249V10.2275Z" fill="#2a5cff"></path><path d="M90.6838 6.3732C86.4786 6.3732 83.4308 8.42451 83.0046 11.8749H87.6975C87.8803 10.6942 88.8549 9.60525 90.6838 9.60525C92.6643 9.60525 93.6398 10.9415 93.6398 12.6518V12.931L89.2821 13.3658C86.0212 13.6769 82.4259 14.6403 82.4259 18.4955C82.4259 21.7594 84.985 23.9981 88.1853 23.9981C91.3856 23.9981 92.7561 22.4753 93.7004 20.454V23.6252H98.3016V12.9938C98.3016 9.20136 95.8333 6.37226 90.6838 6.37226V6.3732ZM93.6398 16.4451C93.6398 19.0877 92.3905 21.139 90.0133 21.139C88.4287 21.139 87.1795 20.1448 87.1795 18.4964C87.1795 16.5997 89.0993 16.2586 90.9888 16.0403L93.6398 15.7601V16.4451Z" fill="#2a5cff"></path><path d="M108.022 10.2275H111.648V6.74523H108.022V2.76725H103.36V6.74616H100.282V10.2284H103.36V18.622C103.36 21.9796 105.798 23.627 109.058 23.627H111.679V20.1457H110.064C108.723 20.1457 108.022 19.7109 108.022 18.249V10.2275Z" fill="#2a5cff"></path><path d="M121.926 6.3732C116.624 6.3732 113.303 10.0101 113.303 15.2016C113.303 20.3931 116.624 23.9991 121.926 23.9991C127.228 23.9991 130.55 20.3622 130.55 15.2016C130.55 10.041 127.228 6.3732 121.926 6.3732ZM121.926 20.7661C119.397 20.7661 118.056 18.6211 118.056 15.2016C118.056 11.7821 119.397 9.60618 121.926 9.60618C124.455 9.60618 125.796 11.7512 125.796 15.2016C125.796 18.652 124.455 20.7661 121.926 20.7661Z" fill="#2a5cff"></path><path d="M143.121 6.3732C140.226 6.3732 138.61 8.08246 137.849 10.1966V6.74616H133.217V23.6261H137.88V14.0199C137.88 11.0353 139.007 9.69896 140.927 9.69896C142.847 9.69896 143.974 11.0353 143.974 13.9581V23.6261H148.637V12.9319C148.637 9.20136 146.991 6.3732 143.121 6.3732H143.121Z" fill="#2a5cff"></path><path d="M157.046 6.74616H152.383V23.6261H157.046V6.74616Z" fill="#2a5cff"></path><path d="M157.137 0H152.323V4.47651H157.137V0Z" fill="#2a5cff"></path><path d="M172.107 17.6268C171.589 19.5853 170.492 20.767 168.572 20.767C166.165 20.767 164.763 18.7148 164.763 15.1716C164.763 11.6284 166.104 9.60712 168.572 9.60712C170.492 9.60712 171.559 11.0371 171.955 12.8092H176.647C175.947 8.86119 172.93 6.37414 168.572 6.37414C163.514 6.37414 160.009 9.88731 160.009 15.1716C160.009 20.4559 163.453 24 168.572 24C172.717 24 176.007 21.7613 176.8 17.6268H172.107Z" fill="#2a5cff"></path><path d="M37.2505 0H28.7188V9.26789H33.533V4.10355H37.3727C41.6082 4.10355 44.0764 6.83894 44.0764 11.813C44.0764 16.7872 41.6082 19.5226 37.3727 19.5226H33.533V14.3666H28.7188V23.6261H37.2505C44.2895 23.6261 49.0431 19.7718 49.0431 11.813C49.0431 3.85428 44.2895 0 37.2505 0Z" fill="#2a5cff"></path><path d="M38.5356 9.26789H33.5376V14.3666H38.5356V9.26789Z" fill="#2a5cff"></path></svg>"""
                        )
                        gr.Markdown(
                            "# Datatonic Analytics Assistant",
                            elem_classes="title right",
                        )

    chatbot = gr.Chatbot()

    with gr.Tab("Ask a question:"):
        # Create a textbox for user questions
        msg = gr.Textbox(show_label=False)

        with gr.Row().style(equal_height=False):
            with gr.Column(scale=3):
                with gr.Row():
                    send_message = gr.Button(
                        value="Submit", variant="primary"
                    ).style(size="sm")
                    clear = gr.ClearButton([msg, chatbot])
                    
    def respond(question, chat_history):
        bot_message = pd_sql_agent(question)
        chat_history.append((question, bot_message))
        time.sleep(2)
        return bot_message, chat_history
    

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    send_message.click(
                respond, [msg, chatbot], [msg, chatbot])
    

demo.launch(share=True)

/var/folders/zk/smbk95753blcq47sdj4f5xkc0000gn/T/ipykernel_8063/1497955290.py:23: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  with gr.Row().style(equal_height=False):
/var/folders/zk/smbk95753blcq47sdj4f5xkc0000gn/T/ipykernel_8063/1497955290.py:28: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  ).style(size="sm")


Running on local URL:  http://127.0.0.1:7871
Running on public URL: https://576662ad372c2b2cad.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new SQLDatabaseSequentialChain chain...


/Users/alvaroazabal/.pyenv/versions/3.11.2/lib/python3.11/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Table names to use:
['orders']

> Entering new SQLDatabaseChain chain...

You are a GoogleSQL expert. Given an input question, first create a syntactically
correct GoogleSQL query to run, then look at the results of the query and return
the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain,
query for at most 100 results using the LIMIT clause as per GoogleSQL. You can
order the results to return the most informative data in the database.

Never query for all columns from a table. You must query only the columns that are
needed to answer the question. Wrap each column name and value in backticks (`)
to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below. Be careful
to not query for columns that do not exist. Also, pay attention to which column
is in which table.

Name all columns in the returned data appropriately. If a column does not have a
matching name in the 